In [1]:
# Libraries:
import pandas as pd
import numpy as np
from __future__ import division
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
%pylab inline
import time
import joblib

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Config:
# In order to display all the columns:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 50

### FUNCTIONS

In [3]:
# Functions to put data in the correct format for estimation:
def get_set_tags(dataframe):
    # Use this function to get the set of tags in the document:
    user_tags=dataframe.usertag
    list_tags=[]
    for i in user_tags:
        for j in i.split(','):
            list_tags.append(j)
    set_tags=list(set(list_tags))
    return set_tags
# removing the null tags:
def remove_objects(list_values,obj="null"):
    return [x for x in list_values if x != obj]

# Functions to separate the usertag columns:
def create_tag_index_dictio(training_df,column="usertag",split_char=","):
    # Creates a dictionary with key: tag and value: list of rows
    tag_series=training_df[column].apply(lambda x:x.split(split_char))
    k=0
    tag_index={}
    for tags_line,index_tag in zip(tag_series,tag_series.index):
        k+=1
        for tag in tags_line:
            if tag != "null":
                if tag not in tag_index.keys():
                    tag_index[tag]=[index_tag]
                else:
                    tag_index[tag].append(index_tag)
        if k%500000==0:
            print "%d completed"%(k)
    return tag_index

def get_tagsDF(dictio_tag_index,training_set):
    # Create the dictionary with key:usertag, value: list with 1 tha impression correspond to this tag 
    N=training_set.shape[0]
    dictio_tags_values={}
    for user_tag in dictio_tag_index.keys():
        sparce_array=[0]*N
        for i in dictio_tag_index[user_tag]:
            sparce_array[i]=1
        dictio_tags_values[user_tag]=sparce_array
    print "%d users completed, creating dataframe"%(len(dictio_tags_values.keys()))
    return pd.DataFrame(dictio_tags_values)

def normalize(dataframe,column="slotprice"):
    dataframe[column]=dataframe[column]/dataframe[column].std()


### Importing datasets:

In [4]:
# Training set:
start_time=time.time()
#filepath="../dataset/train.csv"  ### -----> put the path where is your file
filepath="../dataset/validation.csv"
#filepath="../dataset/test.csv"
data=pd.read_csv(filepath)
print "data imported, num rows: %d, num cols: %d"%(data.shape[0],data.shape[1])


data imported, num rows: 299749, num cols: 26


In [5]:
# validation set:
#filepath_val="../dataset/validation.csv"
#data_val=pd.read_csv(filepath_val)

#validation_set=data_val[data_val["bidprice"]>data_val["payprice"]]
#validation_set=validation_set.reset_index(drop=True)
#print "validation set: %d rows"%(validation_set.shape[0])

In [6]:
training_set=data[data["bidprice"]>data["payprice"]]
training_set=training_set.reset_index(drop=True)
print "training set: %d rows"%(training_set.shape[0])

training set: 295711 rows


In [7]:
# According to the iPINyou paper:
columns_to_dummy=["weekday","hour","region","city","adexchange","slotwidth","slotheight","slotvisibility",
                  "slotformat"]
columns_to_drop=["bidid","logtype","userid","urlid","url","bidprice","payprice","keypage","creative","domain","IP","slotid"]

In [8]:
# Droping columns that we don't use:
training_set.drop(columns_to_drop, axis=1, inplace=True)
print "remaining cols: %d" % (training_set.shape[1])

remaining cols: 14


In [9]:
# Getting the set of tags:
list_tags=get_set_tags(training_set)
list_tags=remove_objects(list_tags)

# creating dictionaries:
dictio_tag_index=create_tag_index_dictio(training_set)
dictio_os_browser=create_tag_index_dictio(training_set,"useragent","_")

# creating new dataframe:
N = training_set.shape[0]
tags_valuesDF=get_tagsDF(dictio_tag_index,training_set)
os_browserDF=get_tagsDF(dictio_os_browser,training_set)

68 users completed, creating dataframe
14 users completed, creating dataframe


In [10]:
# Creating categorical Variables:
training_set=pd.get_dummies(training_set,columns=columns_to_dummy)
normalize(training_set)

In [11]:
# Concatenating
clean_data = pd.concat([training_set, tags_valuesDF,os_browserDF], axis=1)
clean_data.drop(["useragent","usertag"],axis=1,inplace=True)

print("--- %s seconds ---" % (time.time() - start_time))

--- 9.75842809677 seconds ---


In [12]:
#for i in clean_data.columns:
#    if i not in clean_data_val.columns:
#        print "num_values: %d , col_name: %s"%(clean_data[i].sum(),i)
    

In [13]:
#clean_data.to_csv("pp_train_data.csv")

In [14]:
#x=pd.read_csv("pp_train_data.csv")

In [15]:
#normalize(clean_data_val)
#clean_data_val.head()

In [16]:
normalize(clean_data)
clean_data.head()

,click,slotprice,advertiser,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,region_0,region_1,region_2,region_3,region_15,region_27,region_40,region_55,region_65,region_79,region_80,region_94,region_106,region_124,region_134,region_146,...,10133,10138,10140,10142,10145,10146,10147,10148,10149,10684,11092,11278,11379,11423,11512,11576,11632,11680,11724,11944,13042,13403,13496,13678,13776,13800,13866,13874,14273,15398,16593,16617,16661,16706,16751,16753,android,chrome,firefox,ie,ios,linux,mac,maxthon,opera,other,safari,sogou,theworld,windows
0,0,0.000000,3427,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
1,0,3.194711,2997,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,1.895167,3427,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
3,0,1.895167,3427,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0.135369,3386,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [17]:
# Saving one file:
pd14=clean_data[clean_data.advertiser==1458].drop("advertiser",axis=1)
train14=np.array(pd14)
#np.save("data_train_1458",data_train_1458)

In [18]:
train14=np.array(pd14)

In [19]:
joblib.dump(train14, './data_for_CTR/val_data_1458.pkl', compress=True)

['./data_for_CTR/val_data_1458.pkl']

In [20]:
#back14=joblib.load('test_dump.pkl')

In [21]:
#pd14.to_csv("pd_14_val.csv",index=False)